In [1]:
import psycopg2
import pandas as pd
import datacompy
import pyodbc
import useful_functions as use

In [2]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [3]:
business_sql = '''
    SELECT * FROM ONEVIEW.BUSINESS
    '''
ov_business_aff = pd.read_sql_query(business_sql, conn)

In [4]:
provider_sql = '''
    SELECT * FROM ONEVIEW.PROVIDER_AFFILIATION
    '''
ov_provider_aff = pd.read_sql_query(provider_sql, conn)

In [5]:
ov_business_aff['number_of_providers'] = [float(x) if x!=None else 0 for x in ov_business_aff.number_of_providers]

In [6]:
consulting = len(ov_provider_aff[ov_provider_aff.description=='Consulting'].drop_duplicates(subset=['medical_education_number']))
primary = len(ov_provider_aff[ov_provider_aff.primary=='P'].drop_duplicates(subset=['medical_education_number']))
unknown_desc = len(ov_provider_aff[ov_provider_aff.description=='UNKNOWN'].drop_duplicates('medical_education_number'))

In [7]:
active = ov_business_aff[ov_business_aff.status_indicator=='A']
teaching = ov_business_aff[ov_business_aff.teaching_hospital=='Y']
e_prescribe = ov_business_aff[ov_business_aff.electronically_prescribe=='Yes                 ']
independent = ov_business_aff[ov_business_aff.owner_status=='INDEPENDENT']
unknown_owner = ov_business_aff[ov_business_aff.owner_status=='UNKNOWN']
for_profit = ov_business_aff[ov_business_aff.profit_status=='For Profit']
unknown_profit = ov_business_aff[ov_business_aff.profit_status=='UNKNOWN']
name = ov_business_aff[ov_business_aff.name=='UNIVERSITY OF MICHIGAN HOSPITAL']
providers_1 = ov_business_aff[(ov_business_aff.number_of_providers>0)&(ov_business_aff.number_of_providers<3)]
providers_2 = ov_business_aff[(ov_business_aff.number_of_providers>2)&(ov_business_aff.number_of_providers<11)]
providers_3 = ov_business_aff[(ov_business_aff.number_of_providers>10)&(ov_business_aff.number_of_providers<51)]
providers_4 = ov_business_aff[(ov_business_aff.number_of_providers>50)&(ov_business_aff.number_of_providers<501)]
providers_5 = ov_business_aff[(ov_business_aff.number_of_providers>500)]
hospital = ov_business_aff[ov_business_aff.class_of_trade_classification_description=='Hospital']
acute = ov_business_aff[ov_business_aff.class_of_trade_facility_type_description=='Acute Care Hospital']
child = ov_business_aff[ov_business_aff.class_of_trade_specialty_description=='Childrens']

illinois = ov_business_aff[ov_business_aff.physical_state=='IL']
states = ov_business_aff[(ov_business_aff.physical_state=='IL')|(ov_business_aff.physical_state=='CA')]
fips = ov_business_aff[(ov_business_aff.federal_information_processing_standard_county=='031')&(ov_business_aff.federal_information_processing_standard_state=='17')]
chicago = ov_business_aff[ov_business_aff.physical_city=='CHICAGO']

zipcode = ov_business_aff[ov_business_aff.physical_zipcode.str.contains('60608')]
zipcodes = ov_business_aff[(ov_business_aff.physical_zipcode.str.contains('60608'))|(ov_business_aff.physical_zipcode.str.contains('97225'))]

In [8]:
sql = '''
    SELECT DISTINCT A.MEDICAL_EDUCATION_NUMBER
    FROM ONEVIEW.PROVIDER_AFFILIATION A, ONEVIEW.BUSINESS B, ONEVIEW.ZIP_CODE Z, ONEVIEW.METROPOLITAN_STATISTICAL_AREA M
    WHERE
    Z.METROPOLITAN_STATISTICAL_AREA=M.CODE
    AND
    M.NAME = 'ABILENE, TX'
    AND
    SUBSTR(B.PHYSICAL_ZIPCODE,1,5) = Z.ZIP_CODE
    AND
    B.ID = A.BUSINESS
    '''
msa = pd.read_sql_query(sql, conn)
msa_count = len(msa)


In [9]:
def test_business_attribute(filtered):
    count = len(ov_provider_aff[ov_provider_aff.business.isin(filtered.id)].drop_duplicates(subset=['medical_education_number']))
    return count

In [10]:
iqvia_list = [active,
 teaching,
 e_prescribe, 
 independent,
 unknown_owner, 
 for_profit,
 unknown_profit,
 name,
 providers_1,
 providers_2,
 providers_3,
 providers_4,
 providers_5,
 hospital,
 acute,
 child,
 illinois,
 states,
 fips,
 chicago,
 zipcode,
 zipcodes]

In [11]:
filter_counts = []
for check in iqvia_list:
    count = test_business_attribute(check)
    filter_counts.append(count)

In [12]:
chunk_1 = filter_counts[:16]
chunk_2 = [primary, consulting]
chunk_3 = filter_counts[16:20]
chunk_4 = [msa_count, filter_counts[20],0,filter_counts[21]]
chunk_1+chunk_2+chunk_3+chunk_4

[818387,
 568808,
 808619,
 65486,
 0,
 145138,
 6234,
 4074,
 7294,
 27580,
 61319,
 430278,
 519334,
 809719,
 801157,
 118130,
 389979,
 1039,
 37082,
 135170,
 22267,
 14182,
 328,
 274,
 0,
 2175]

In [65]:
msa = ov_business_aff[(ov_business_aff.metropolitan_statistical_area=='10180')]
ov_provider_aff[(ov_provider_aff.business.isin(msa.id))&(~ov_provider_aff.medical_education_number.isin(list(msa_count.medical_education_number)))]

,id,business,medical_education_number,type,description,primary,rank,group,group_description,batch_business_date
896127,INS000038444955067031,INS00003844,49550670312,3,Admitting,P,9,ADM,Admitting,2021-04-23
968015,INS000802350487881027,INS00080235,04878810274,4,Staff,A,6,LTC,Long Term Care,2021-04-23
969659,INS000802354955067031,INS00080235,49550670312,4,Staff,P,10,LTC,Long Term Care,2021-04-23
970350,INS000802374951165011,INS00080237,49511650118,4,Staff,A,6,LTC,Long Term Care,2021-04-23
973753,INS000802344951165011,INS00080234,49511650118,4,Staff,A,6,LTC,Long Term Care,2021-04-23
1021900,INS000379730481290015,INS00037973,04812900159,6,Treating,A,8,LTC,Long Term Care,2021-04-23
1257167,INS000038444220111289,INS00003844,42201112898,8,Admitting (Inferred),A,7,ADM,Admitting,2021-04-23
1260232,INS000038440240593006,INS00003844,02405930067,8,Admitting (Inferred),A,7,ADM,Admitting,2021-04-23


In [66]:
ov_business_aff[ov_business_aff.id.isin(['INS00003844','INS00080235','INS00080237','INS00080234','INS0003797','INS00003844'])]

,id,name,doing_business_as,iqvia_address_id,physical_address_1,physical_address_2,physical_city,physical_state,physical_zipcode,postal_address_1,...,federal_information_processing_standard_state,federal_information_processing_standard_county,number_of_providers,electronic_medical_record,electronically_prescribe,pay_for_performance,deactivation_reason,replacement_business,status_indicator,batch_business_date
3068,INS00003844,ANSON GENERAL HOSPITAL,None,0001994672,101 AVENUE J,None,ANSON,TX,79501-2113,101 AVENUE J,...,48,253,4.0,Yes,Yes,No,None,None,A,2021-04-23
67164,INS00080234,HOMEPLACE MANOR,None,0005289873,425 SW AVENUE F,None,HAMLIN,TX,79520-4615,425 SW AVENUE F,...,48,253,1.0,Yes,Yes,No,None,None,A,2021-04-23
67165,INS00080235,STAMFORD RESIDENCE AND REHABILITATION CENTER,None,0005086495,1003 COLUMBIA ST,None,STAMFORD,TX,79553-6825,1003 COLUMBIA ST,...,48,253,2.0,Yes,Yes,None,None,None,A,2021-04-23
67167,INS00080237,VALLEY VIEW CARE CENTER,None,0005258148,101 LIBERTY LN,None,ANSON,TX,79501-2105,101 LIBERTY LN,...,48,253,1.0,Yes,Yes,No,None,None,A,2021-04-23


In [68]:
zips[zips.zip_code.isin(['79501','79520','79553'])]

,id,zip_code,state,city,type,county_federal_information_processing,latitude,longitude,metropolitan_statistical_area,primary_metropolitan_statistical_area
67119,79501 ANSON,79501,TX,ANSON,,48253,32.7418,99.89,,
67120,79501 RADIUM,79501,TX,RADIUM,,48253,32.7418,99.89,,
67138,79520 HAMLIN,79520,TX,HAMLIN,,48253,32.8763,100.1229,,
67179,79553 STAMFORD,79553,TX,STAMFORD,,48253,32.9352,99.8038,,
67180,79553 TUXEDO,79553,TX,TUXEDO,,48253,32.9352,99.8038,,


In [ ]:
sql = '''
    SELECT *
    FROM ONEVIEW.ZIP_CODE Z, ONEVIEW.METROPOLITAN_STATISTICAL_AREA M
    WHERE
    Z.METROPOLITAN_STATISTICAL_AREA=M.CODE
    AND
    M.NAME = 'ABILENE, TX'
    AND
    SUBSTR(B.PHYSICAL_ZIPCODE,1,5) = Z.ZIP_CODE
    AND
    B.ID = A.BUSINESS
    '''
msa_count = pd.read_sql_query(sql, conn)

In [93]:
class_list = list(ov_business_aff.class_of_trade_classification_description.unique())

In [95]:
classifications = []
for desc in class_list:
    this = ov_business_aff[ov_business_aff.class_of_trade_classification_description==desc]
    count = test_business_attribute(this)
    new_dict= {
        'CLASSIFICATION': desc,
        'COUNT': count
    }
    classifications.append(new_dict)

In [97]:
type_list = list(ov_business_aff.class_of_trade_facility_type_description.unique())
types = []
for desc in type_list:
    this = ov_business_aff[ov_business_aff.class_of_trade_facility_type_description==desc]
    count = test_business_attribute(this)
    new_dict= {
        'TYPE': desc,
        'COUNT': count
    }
    types.append(new_dict)

In [ ]:
acute = ov_business_aff[ov_business_aff.class_of_trade_facility_type_description=='Acute Care Hospital']

In [96]:
classifications

[{'CLASSIFICATION': 'Buying Organization', 'COUNT': 0},
 {'CLASSIFICATION': 'Health Alliance', 'COUNT': 0},
 {'CLASSIFICATION': 'Corporate Parent', 'COUNT': 0},
 {'CLASSIFICATION': 'Residential', 'COUNT': 56943},
 {'CLASSIFICATION': 'Outpatient Center', 'COUNT': 0},
 {'CLASSIFICATION': 'Miscellaneous', 'COUNT': 0},
 {'CLASSIFICATION': 'Hospital', 'COUNT': 809719},
 {'CLASSIFICATION': 'Correctional Facility', 'COUNT': 0},
 {'CLASSIFICATION': 'Laboratory', 'COUNT': 0},
 {'CLASSIFICATION': 'Emergency', 'COUNT': 0},
 {'CLASSIFICATION': 'Research', 'COUNT': 0},
 {'CLASSIFICATION': 'Other', 'COUNT': 0},
 {'CLASSIFICATION': 'Work place Wellness', 'COUNT': 0},
 {'CLASSIFICATION': 'Pharmacy', 'COUNT': 0},
 {'CLASSIFICATION': 'Elder Care', 'COUNT': 0},
 {'CLASSIFICATION': 'Insurance', 'COUNT': 0},
 {'CLASSIFICATION': 'Supply', 'COUNT': 0},
 {'CLASSIFICATION': 'Veterinary Medicine', 'COUNT': 0},
 {'CLASSIFICATION': 'Academic', 'COUNT': 0},
 {'CLASSIFICATION': 'Healthcare Information Technology', 

In [100]:
TYPE = pd.DataFrame(types)
TYPE[TYPE.COUNT!=0]

,TYPE,COUNT
8,Acute Care Hospital,801157
9,Rehabilitation Hospital,7293
10,Psychiatric Hospital,13187
15,Nursing Homes,56943
